In [0]:
df = spark.table("teams.data_science.pp_churn_features_v3")

In [0]:
all_feature_names = ['unique_levels_played', 'market_idx','dayofweek','rounds_played', 'avg_attempts', 'total_attempts', 'avg_moves', 'win_rate', 'assist_success_rate', 'unassist_success_rate', 'assist_rate', 'total_boosters_used', 'total_boosters_spent', 'used_boosters_rate', 'spend_boosters_rate', 'avg_difficulty_score', 'rate_hard_levels', 'rate_superhard_levels', 'min_room_id_int', 'max_room_id_int', 'daily_win_rate_ref', 'daily_avg_boosters_used_ref', 'daily_avg_boosters_spent_ref', 'attribution_source_cd_idx', 'country_cd_idx', 'payer_type_cd_idx', 'iap_lifetime_amt', 'days_since_install', 'days_since_last_purchase', 'ad_revenue_amt', 'iap_revenue_amt', 'session_qty', 'total_session_length_qty', 'avg_session_length', 'sessions_per_round', 'avg_population_wr_on_levels_played_today', 'avg_population_assisted_rate_today', 'avg_population_attempts_today', 'wr_diff_vs_population', 'attempts_diff_vs_population', 'assist_rate_diff_vs_population', 'active_days_l7d', 'total_rounds_l7d', 'avg_rounds_l7d', 'avg_win_rate_l7d', 'avg_attempts_l7d', 'boosters_used_l7d', 'boosters_spent_l7d', 'avg_used_boosters_rate_l7d', 'active_days_l14d', 'avg_rounds_l14d', 'avg_win_rate_l14d', 'std_rounds_l14d', 'std_win_rate_l14d', 'active_days_l30d', 'avg_rounds_l30d', 'rounds_trend_weekly', 'win_rate_trend_weekly', 'boosters_usage_trend_weekly', 'rounds_ratio_7d_vs_14_7d', 'frequency_ratio_7d_vs_14d', 'levels_progressed_l7d', 'levels_progressed_l14d', 'levels_progressed_l30d', 'days_on_current_max_level', 'level_diversity_ratio',] 


In [0]:
# split into payers and non-payers:
#payers = df.filter(df.iap_lifetime_amt > 0)
#non_payers = df.filter(df.iap_lifetime_amt == 0)

In [0]:
historical = ['active_days_l30d', 'avg_rounds_l30d','levels_progressed_l30d','attribution_source_cd_idx', 'country_cd_idx', 'total_session_length_qty','assist_rate_diff_vs_population','attemps_diff_vs_population','wr_diff_vs_population','avg_session_length','max_room_id_int']




In [0]:
%sql describe teams.data_science.pp_churn_features_v3

In [0]:
###select more historical, categorical columns of data
['market_idx','dayofweek','rounds_played', 'avg_attempts', 'total_attempts', 'avg_moves', 'win_rate', 'assist_success_rate', 'unassist_success_rate', 'assist_rate', 'total_boosters_used', 'total_boosters_spent', 'used_boosters_rate', 'spend_boosters_rate', 'avg_difficulty_score', 'rate_hard_levels', 'rate_superhard_levels', 'min_room_id_int', 'max_room_id_int', 'daily_win_rate_ref', 'daily_avg_boosters_used_ref', 'daily_avg_boosters_spent_ref', 'attribution_source_cd_idx', 'country_cd_idx', 'payer_type_cd_idx', 'iap_lifetime_amt', 'days_since_install', 'days_since_last_purchase', 'ad_revenue_amt', 'iap_revenue_amt', 'session_qty', 'total_session_length_qty', 'avg_session_length', 'sessions_per_round', 'avg_population_wr_on_levels_played_today', 'avg_population_assisted_rate_today', 'avg_population_attempts_today', 'wr_diff_vs_population', 'attempts_diff_vs_population', 'assist_rate_diff_vs_population', 'active_days_l7d', 'total_rounds_l7d', 'avg_rounds_l7d', 'avg_win_rate_l7d', 'avg_attempts_l7d', 'boosters_used_l7d', 'boosters_spent_l7d', 'avg_used_boosters_rate_l7d', 'active_days_l14d', 'avg_rounds_l14d', 'avg_win_rate_l14d', 'std_rounds_l14d', 'std_win_rate_l14d', 'active_days_l30d', 'avg_rounds_l30d', 'rounds_trend_weekly', 'win_rate_trend_weekly', 'boosters_usage_trend_weekly', 'rounds_ratio_7d_vs_14_7d', 'frequency_ratio_7d_vs_14d', 'levels_progressed_l7d', 'levels_progressed_l14d', 'levels_progressed_l30d', 'days_on_current_max_level', 'level_diversity_ratio',] 

In [0]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler


assembler_corr = VectorAssembler(
    inputCols=all_feature_names, 
    outputCol="features_corr",
    handleInvalid="skip"
)
df_vector = assembler_corr.transform(df).select("features_corr")


matrix = Correlation.corr(df_vector, "features_corr", "pearson").head()

corr_matrix = matrix[0].toArray()



In [0]:
import numpy as np

# 3. El umbral de correlación para considerar la eliminación
THRESHOLD = 0.95

# --------------------

print(f"Analizando {len(all_feature_names)} features para alta correlación (umbral={THRESHOLD})...")

# Set para guardar los nombres de las features a eliminar
features_to_drop = set()

# Obtener el número de features
n_features = len(all_feature_names)

# Calcular la correlación absoluta media para cada feature
# Esta será nuestra heurística para decidir cuál eliminar
mean_abs_corr = np.mean(np.abs(corr_matrix), axis=1)

# Iterar solo sobre el triángulo superior de la matriz
for i in range(n_features):
    for j in range(i + 1, n_features):
        
        feature_i = all_feature_names[i]
        feature_j = all_feature_names[j]

        # Si alguna de las features ya está marcada para eliminarse, saltar este par
        if feature_i in features_to_drop or feature_j in features_to_drop:
            continue
            
        correlation = corr_matrix[i, j]

        # Comprobar si la correlación absoluta supera el umbral
        if abs(correlation) > THRESHOLD:
            
            # Tenemos un par altamente correlacionado.
            # Decidimos cuál eliminar basándonos en la correlación media.
            # El que tenga la media más alta es "más redundante".
            
            if mean_abs_corr[i] > mean_abs_corr[j]:
                features_to_drop.add(feature_i)
                print(f"🚨 Par encontrado: ('{feature_i}', '{feature_j}') -> Corr: {correlation:.2f}. ")
                print(f"   -> Marcando '{feature_i}' para eliminar (más redundante).")
            else:
                features_to_drop.add(feature_j)
                print(f"🚨 Par encontrado: ('{feature_i}', '{feature_j}') -> Corr: {correlation:.2f}. ")
                print(f"   -> Marcando '{feature_j}' para eliminar (más redundante).")

print("\n--- Resultado ---")
print(f"Total de features a eliminar: {len(features_to_drop)}")
print(features_to_drop)

In [0]:
# Obtener la lista final de features
final_feature_names = [name for name in all_feature_names if name not in features_to_drop]

print(f"\nFeatures originales: {len(all_feature_names)}")
print(f"Features eliminadas: {len(features_to_drop)}")
print(f"Features finales: {len(final_feature_names)}")

# Ahora, crea tu VectorAssembler para el modelo SOLO con 'final_feature_names'

# K-MEANS

In [0]:
# --- STEP 1: Data Preparation ---

from pyspark.ml.feature import VectorAssembler, StandardScaler, Imputer
from pyspark.ml import Pipeline

# 1. Select key features for clustering
# We choose features representing Engagement, Skill, Monetization, and Progression.
features_for_clustering = [
    'avg_rounds_l7d',
    'active_days_l7d',
    'avg_win_rate_l7d',
    'avg_attempts_l7d',
    'boosters_used_l7d',
    'iap_lifetime_amt',
    'ad_revenue_amt',
    'levels_progressed_l7d',
    'days_on_current_max_level',
    'avg_session_length',
    'days_since_install',
    'days_since_last_purchase',
    'session_qty',
    'active_days_l14d',
    'active_days_l30d',
    'market_idx',
    'dayofweek',
    'attribution_source_cd_idx',
    'country_cd_idx'
]

# 2. Create imputation stage
# K-Means cannot handle nulls. We'll use median to fill missing values.
imputer = Imputer(
    inputCols=features_for_clustering,
    outputCols=[f"{c}_imputed" for c in features_for_clustering],
    strategy="median"
)

# 3. Create vector assembler stage
# This will use the *new* imputed columns
imputed_cols = [f"{c}_imputed" for c in features_for_clustering]
assembler = VectorAssembler(
    inputCols=imputed_cols,
    outputCol="unscaled_features"
)

# 4. Create scaling stage
# K-Means uses Euclidean distance, so scaling is critical.
scaler = StandardScaler(
    inputCol="unscaled_features",
    outputCol="features", # This is the final column for the model
    withStd=True,
    withMean=True
)

# 5. Build the preparation pipeline
preprocessing_pipeline = Pipeline(stages=[imputer, assembler, scaler])

# 6. Fit and transform the data
# 'df' is your source DataFrame
try:
    preprocessing_model = preprocessing_pipeline.fit(df)
    scaled_data = preprocessing_model.transform(df)
    
    print("Data preparation successful.")
    print("DataFrame now contains a 'features' column (scaled) and an 'unscaled_features' column.")
    
    # Show a sample for verification
    scaled_data.select("features", "unscaled_features").show(5, truncate=False)

except Exception as e:
    print(f"An error occurred during data preparation: {e}")

In [0]:
# --- STEP 2: Find Optimal K (Elbow Method) ---

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import pandas as pd
import matplotlib.pyplot as plt

# 1. Define the range of K to test
# We'll test from K=2 (minimum clusters) up to K=15
k_values = range(7, 18)

# 2. List to store the cost (WCSS) for each K
wcss_values = []

print("Starting K-Means training for Elbow Method...")
print(f"Testing K values from {min(k_values)} to {max(k_values)}.")

# 3. Loop over each K, train model, and get cost
# This will use the 'scaled_data' DataFrame from Step 1
evaluator = ClusteringEvaluator(featuresCol='features')

for k in k_values:
    try:
        kmeans = KMeans(featuresCol='features', k=k, seed=1)
        model = kmeans.fit(scaled_data)
        
        # Calculate WCSS
        # Evaluator computes silhouette, which is WCSS for squaredEuclidean
        wcss = model.summary.trainingCost
        wcss_values.append(wcss)
        
        print(f"  K={k}, WCSS={wcss:.2f}")
        
    except Exception as e:
        print(f"Failed for K={k}: {e}")
        wcss_values.append(None) # Append None to keep lists aligned

print("Elbow method training complete.")

# 4. Prepare data for plotting
# Convert results to a Pandas DataFrame for easy plotting
elbow_data = pd.DataFrame({
    'K': k_values,
    'WCSS': wcss_values
}).dropna() # Drop any K that failed

# 5. Plot the Elbow curve
print("Generating Elbow Plot...")

plt.figure(figsize=(10, 6))
plt.plot(elbow_data['K'], elbow_data['WCSS'], 'bx-')
plt.xlabel('Number of clusters (K)')
plt.ylabel('WCSS (Training Cost)')
plt.title('K-Means Elbow Method')
plt.xticks(elbow_data['K'])
plt.grid(True)
display(plt.gcf()) 

In [0]:
# --- STEP 3: Train Final Model ---

from pyspark.ml.clustering import KMeans

# 1. Define the optimal K based on our analysis
OPTIMAL_K = 15

print(f"Training final K-Means model with K={OPTIMAL_K}...")

# 2. Initialize the model
kmeans_final = KMeans(featuresCol='features', k=OPTIMAL_K, seed=1)

# 3. Fit the model on the scaled data
# This creates our final, trained clustering model
final_model = kmeans_final.fit(scaled_data)

# 4. Assign clusters to all players
# This adds a new column 'prediction' (our cluster ID)
data_with_clusters = final_model.transform(scaled_data)

print("Model training complete.")
print("Added 'prediction' column with cluster IDs to the DataFrame.")

# 5. Show a sample of the results
# We select the original features (unscaled) and the new prediction
original_features_plus_cluster = [
    'avg_rounds_l7d',
    'active_days_l7d',
    'avg_win_rate_l7d',
    'boosters_used_l7d',
    'iap_lifetime_amt',
    'prediction'
]

data_with_clusters.select(original_features_plus_cluster).show(20)

In [0]:
# --- STEP 4: Profile and Analyze Clusters ---

import pyspark.sql.functions as F

# 1. Get the list of imputed column names from Step 1
# (This assumes 'imputer' is still in memory from Step 1)
try:
    imputed_cols = imputer.getOutputCols()
    original_cols = features_for_clustering # List from Step 1
except NameError:
    # Fallback in case the notebook state was lost
    print("Re-defining feature lists...")
    original_cols = [
        'avg_rounds_l7d', 'active_days_l7d', 'avg_win_rate_l7d', 'avg_attempts_l7d',
        'boosters_used_l7d', 'iap_lifetime_amt', 'ad_revenue_amt', 'levels_progressed_l7d',
        'days_on_current_max_level', 'avg_session_length', 'days_since_install',
        'days_since_last_purchase', 'session_qty', 'active_days_l14d', 'active_days_l30d'
    ]
    imputed_cols = [f"{c}_imputed" for c in original_cols]


# 2. Create a list of aggregation expressions
# We calculate the mean for each imputed feature and alias it back to the original name
agg_expressions = []
for original, imputed in zip(original_cols, imputed_cols):
    agg_expressions.append(
        F.mean(imputed).alias(original)
    )

# 3. Add an expression to count the size of each cluster
agg_expressions.append(
    F.count("*").alias("cluster_size")
)

print("Calculating cluster profiles...")

# 4. Group by cluster, apply aggregations, and order
# This gives us the full profile for all 10 clusters
cluster_profiles_spark = data_with_clusters \
    .groupBy("prediction") \
    .agg(*agg_expressions) \
    .orderBy("prediction")

# 5. Convert to Pandas for easier analysis and viewing
cluster_profiles_pandas = cluster_profiles_spark.toPandas()

print("Cluster profiling complete.")

# 6. Display the profiles
# This table is the "answer" - it tells us what each cluster means.
display(cluster_profiles_pandas)

In [0]:
# --- STEP 5: Visualize Cluster Profiles ---
# (Assuming 'cluster_profiles_pandas' is our Pandas DataFrame from Step 4)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import pandas as pd # Ensure pandas is imported if not already

# --- Visualization 1: Cluster Sizes (Bar Chart) ---

print("Generating Cluster Size Bar Chart...")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=cluster_profiles_pandas, 
    x='prediction', 
    y='cluster_size',
    palette='viridis' # A nice color palette
)
plt.title('Distribution of Players per Cluster (Cluster Size)')
plt.xlabel('Cluster ID (prediction)')
plt.ylabel('Number of Players')
plt.xticks(rotation=0)
display(plt.gcf()) # Use display() in Databricks

# --- Visualization 2: Feature Profiles (Heatmap) ---

print("Generating Normalized Feature Profile Heatmap...")

# 1. Prepare data: Set index and drop non-feature column
# We need to scale only the 15 features, not 'cluster_size'
profiles_to_scale = cluster_profiles_pandas.set_index('prediction')
features_only = profiles_to_scale.drop(columns=['cluster_size'])

# 2. Scale the data (column-wise)
# Each feature (column) will be scaled from 0 (min) to 1 (max)
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features_only)

# 3. Recreate DataFrame with scaled values
scaled_profiles_df = pd.DataFrame(
    scaled_features,
    index=features_only.index,
    columns=features_only.columns
)

# 4. Plot the heatmap
plt.figure(figsize=(16, 10)) # Make it large for readability
sns.heatmap(
    scaled_profiles_df, 
    annot=True,     # Show the scaled values (0.0 to 1.0)
    fmt=".2f",      # Format to 2 decimal places
    cmap='coolwarm', # Blue (low) to Red (high) is very intuitive
    linewidths=0.5
)
plt.title('Normalized Feature Averages by Cluster')
plt.xlabel('Features')
plt.ylabel('Cluster ID (prediction)')
plt.gcf()

In [0]:
%sql
select churn3, count(1) from teams.data_science.pp_churn_features_v3_clusters where cluster_6=1 group by 1

In [0]:
%sql
select churn3, count(1) from teams.data_science.pp_churn_features_v3_clusters where cluster_8=1 group by 1

In [0]:
# --- STEP 6: Cross-reference Clusters with Churn3 ---

import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Calculate churn rate per cluster
print("Calculating churn rate per cluster...")

# We use the 'data_with_clusters' DataFrame from Step 3
# It should contain the original 'churn3' column
churn_by_cluster_spark = data_with_clusters \
    .groupBy("prediction") \
    .agg(
        F.avg("churn3").alias("churn_rate"),
        F.count("*").alias("cluster_size")
    ) \
    .orderBy(F.desc("churn_rate"))

# 2. Convert to Pandas for analysis and plotting
churn_by_cluster_pandas = churn_by_cluster_spark.toPandas()

# 3. Display the churn rate table
print("Churn Rate by Cluster (Highest to Lowest):")
display(churn_by_cluster_pandas)




print("\nGenerando Gráfico de Tasa de Churn (Paleta Corregida)...")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=churn_by_cluster_pandas,
    x='prediction',
    y='churn_rate',
    palette='coolwarm' # 
)
plt.title('Tasa de Churn3 por Cluster de Jugador')
plt.xlabel('Cluster ID (prediction)')
plt.ylabel('Tasa de Churn3 (Avg)')
plt.axhline(y=0.03, color='black', linestyle='--', label='Churn Promedio (3%)')
plt.legend()
display(plt.gcf())

In [0]:
# --- Show a Sample of Cluster 6 Players ---

import pyspark.sql.functions as F

# 1. Define the features we want to inspect
# These are the 15 features from our clustering
features_to_show = [
    'avg_rounds_l7d', 'active_days_l7d', 'avg_win_rate_l7d', 'avg_attempts_l7d',
    'boosters_used_l7d', 'iap_lifetime_amt', 'ad_revenue_amt', 'levels_progressed_l7d',
    'days_on_current_max_level', 'avg_session_length', 'days_since_install',
    'days_since_last_purchase', 'session_qty', 'active_days_l14d', 'active_days_l30d',
    'churn3', # Also show the target variable
    'prediction' # And the cluster ID
]

print("Displaying a sample of players from Cluster 6 (High Churn Group)...")

# 2. Filter the DataFrame for Cluster 6 and select our columns
cluster_6_sample = data_with_clusters \
    .filter(F.col("prediction") == 6) \
    .select(features_to_show)

# 3. Show the sample
# truncate=False ensures we see the full values in the columns
cluster_6_sample.show(20, truncate=False)

In [0]:
display(cluster_6_sample)

In [0]:
# --- Build Model Pipeline Using Cluster as a Feature ---

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
# Asegúrate de importar tu clasificador de XGBoost
# from pyspark.ml.classification import XGBoostClassifier 

# 1. List of your original 56 feature columns
# (Asegúrate de que esta lista esté definida)
original_56_features = all_feature_names

# 2. Stage 1: StringIndexer for the cluster ID
# Converts the 'prediction' column (e.g., 6.0) into a categorical index
cluster_indexer = StringIndexer(
    inputCol="prediction",
    outputCol="prediction_idx",
    handleInvalid="keep" # Keep unseen labels
)

# 3. Stage 2: OneHotEncoder for the indexed cluster ID
# Creates a sparse vector (e.g., [0,0,0,0,0,0,1,0,0,0])
cluster_ohe = OneHotEncoder(
    inputCols=["prediction_idx"],
    outputCols=["cluster_ohe_vec"]
)

# 4. Stage 3: Final VectorAssembler
# This combines your ORIGINAL features with your NEW cluster features
final_feature_list = original_56_features + ["cluster_ohe_vec"]

final_assembler = VectorAssembler(
    inputCols=final_feature_list,
    outputCol="features", # This is the final vector for XGBoost
    handleInvalid="skip" # O usa tu 'Imputer' si lo tienes en un pipeline
)



In [0]:
cluster_ohe

In [0]:
# --- Materialize Table with OHE Clusters ---

from pyspark.sql.functions import lit, col

# 1. Define all original columns to keep
# We get all columns from your DataFrame EXCEPT the ones we created for clustering
# (Esto asume que no has borrado ninguna columna todavía)
original_cols_to_keep = [
    c for c in data_with_clusters.columns 
    if c not in ('unscaled_features', 'features', 'prediction')
]

# 2. Define the list of cluster IDs (0 to 9)
# Esto es para decirle al pivot qué columnas crear
num_clusters = 10
cluster_ids = list(range(num_clusters)) # [0, 1, 2, ..., 9]

print(f"Pivoting 'prediction' column into {num_clusters} new OHE columns...")

# 3. Group by original columns, pivot on 'prediction', and fill
materialized_df = data_with_clusters \
    .groupBy(original_cols_to_keep) \
    .pivot("prediction", cluster_ids) \
    .agg(lit(1)) \
    .fillna(0)

# 4. Rename new columns for clarity
# El pivot crea columnas con nombres "0", "1", "2"...
# Las renombramos a "cluster_0", "cluster_1", "cluster_2"...
for i in cluster_ids:
    materialized_df = materialized_df.withColumnRenamed(
        str(i), f"cluster_{i}"
    )

print("DataFrame materializado con éxito.")
materialized_df.select(original_cols_to_keep[-5:] + [f"cluster_{i}" for i in cluster_ids]).show(10)

In [0]:
display(materialized_df.limit(19))

In [0]:
materialized_df.createOrReplaceTempView("vo_clusters")

In [0]:
%sql
create table teams.data_science.pp_churn_features_v3_clusters
using delta
as select * from vo_clusters

In [0]:
%sql
select count(1) from teams.data_science.pp_churn_features_v3_clusters

# V2

In [0]:
# --- STEP 5: Visualize Cluster Profiles ---
# (Assuming 'cluster_profiles_pandas' is our Pandas DataFrame from Step 4)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import pandas as pd # Ensure pandas is imported if not already

# --- Visualization 1: Cluster Sizes (Bar Chart) ---

print("Generating Cluster Size Bar Chart...")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=cluster_profiles_pandas, 
    x='prediction', 
    y='cluster_size',
    palette='viridis' # A nice color palette
)
plt.title('Distribution of Players per Cluster (Cluster Size)')
plt.xlabel('Cluster ID (prediction)')
plt.ylabel('Number of Players')
plt.xticks(rotation=0)
display(plt.gcf()) # Use display() in Databricks

# --- Visualization 2: Feature Profiles (Heatmap) ---

print("Generating Normalized Feature Profile Heatmap...")

# 1. Prepare data: Set index and drop non-feature column
# We need to scale only the 15 features, not 'cluster_size'
profiles_to_scale = cluster_profiles_pandas.set_index('prediction')
features_only = profiles_to_scale.drop(columns=['cluster_size'])

# 2. Scale the data (column-wise)
# Each feature (column) will be scaled from 0 (min) to 1 (max)
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features_only)

# 3. Recreate DataFrame with scaled values
scaled_profiles_df = pd.DataFrame(
    scaled_features,
    index=features_only.index,
    columns=features_only.columns
)

# 4. Plot the heatmap
plt.figure(figsize=(16, 10)) # Make it large for readability
sns.heatmap(
    scaled_profiles_df, 
    annot=True,     # Show the scaled values (0.0 to 1.0)
    fmt=".2f",      # Format to 2 decimal places
    cmap='coolwarm', # Blue (low) to Red (high) is very intuitive
    linewidths=0.5
)
plt.title('Normalized Feature Averages by Cluster')
plt.xlabel('Features')
plt.ylabel('Cluster ID (prediction)')
plt.gcf()

In [0]:
# --- STEP 6: Cross-reference Clusters with Churn3 ---

import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Calculate churn rate per cluster
print("Calculating churn rate per cluster...")

# We use the 'data_with_clusters' DataFrame from Step 3
# It should contain the original 'churn3' column
churn_by_cluster_spark = data_with_clusters \
    .groupBy("prediction") \
    .agg(
        F.avg("churn3").alias("churn_rate"),
        F.count("*").alias("cluster_size")
    ) \
    .orderBy(F.desc("churn_rate"))

# 2. Convert to Pandas for analysis and plotting
churn_by_cluster_pandas = churn_by_cluster_spark.toPandas()

# 3. Display the churn rate table
print("Churn Rate by Cluster (Highest to Lowest):")
display(churn_by_cluster_pandas)




print("\nGenerando Gráfico de Tasa de Churn (Paleta Corregida)...")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=churn_by_cluster_pandas,
    x='prediction',
    y='churn_rate',
    palette='coolwarm' # 
)
plt.title('Tasa de Churn3 por Cluster de Jugador')
plt.xlabel('Cluster ID (prediction)')
plt.ylabel('Tasa de Churn3 (Avg)')
plt.axhline(y=0.03, color='black', linestyle='--', label='Churn Promedio (3%)')
plt.legend()
display(plt.gcf())

In [0]:
# --- Materialize Table with OHE Clusters ---

from pyspark.sql.functions import lit, col

# 1. Define all original columns to keep
# We get all columns from your DataFrame EXCEPT the ones we created for clustering
# (Esto asume que no has borrado ninguna columna todavía)
original_cols_to_keep = [
    c for c in data_with_clusters.columns 
    if c not in ('unscaled_features', 'features', 'prediction')
]

# 2. Define the list of cluster IDs (0 to 9)
# Esto es para decirle al pivot qué columnas crear
num_clusters = 15
cluster_ids = list(range(num_clusters)) # [0, 1, 2, ..., 9]

print(f"Pivoting 'prediction' column into {num_clusters} new OHE columns...")

# 3. Group by original columns, pivot on 'prediction', and fill
materialized_df = data_with_clusters \
    .groupBy(original_cols_to_keep) \
    .pivot("prediction", cluster_ids) \
    .agg(lit(1)) \
    .fillna(0)

# 4. Rename new columns for clarity
# El pivot crea columnas con nombres "0", "1", "2"...
# Las renombramos a "cluster_0", "cluster_1", "cluster_2"...
for i in cluster_ids:
    materialized_df = materialized_df.withColumnRenamed(
        str(i), f"cluster_{i}"
    )

print("DataFrame materializado con éxito.")
materialized_df.select(original_cols_to_keep[-5:] + [f"cluster_{i}" for i in cluster_ids]).show(10)

In [0]:
materialized_df.createOrReplaceTempView("vo_clusters")

In [0]:
%sql
select * from teams.data_science.pp_churn_features_v3_clusters

In [0]:
%sql

create table teams.data_science.pp_churn_features_v3_clusters
using delta
as select * from vo_clusters

In [0]:
%sql
select * from teams.data_science.pp_churn_features_v3_clusters_15